In [3]:
print("Hello")

Hello


In [4]:
zipfile_path = 'Birds400.zip'
path_to_data = './data'

In [ ]:
import zipfile
with zipfile.ZipFile(zipfile_path, 'r') as zip_ref:
    zip_ref.extractall(path_to_data)

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
import wandb
from torch.utils.data import DataLoader,SubsetRandomSampler
from torchvision.models import resnet18,resnet34,resnet50,resnet101,resnet152

In [ ]:
train_path = path_to_data + '/Birds400/train'
test_path = path_to_data + '/Birds400/test'
val_path = path_to_data + '/Birds400/valid'

train_data = ImageFolder(train_path,transform=transforms.ToTensor())
test_data = ImageFolder(test_path,transform=transforms.ToTensor())
val_data = ImageFolder(val_path,transform=transforms.ToTensor())

In [ ]:
torch.backends.cudnn.benchmarks = True

torch.backends.cudnn.deterministic = True

wandb.login(key='46f2d6a5ffcc458fed2cca6cf446900f97e396e1')

In [ ]:
config=dict(
IMAGE_SIZE = 128,
BATCH_SIZE = 64,
EPOCHS = 10,
LEARNING_RATE = 3e-5,
N_WORKER = 4,
PIN_MEMORY = True,
Model='ResNet50',
saved_path = './Birds400/saved'
)

In [ ]:
wandb.init(project='Bird_Classifier',config=config)
config = wandb.config

In [ ]:
def Shuffler(n):
    index = np.random.permutation(n)
    return index

In [ ]:
train_indices = Shuffler(len(train_data))
val_indices = Shuffler(len(val_data))
print("Training Indices :- ",train_indices)
print("Val Indices :- ",val_indices)

In [ ]:
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(train_data,batch_size=config.BATCH_SIZE,sampler=train_sampler,num_workers=config.N_WORKER,pin_memory=config.PIN_MEMORY)
val_loader = DataLoader(val_data,batch_size=config.BATCH_SIZE,sampler=val_sampler,num_workers=config.N_WORKER,pin_memory=config.PIN_MEMORY)

In [ ]:
if config.Model=='ResNet18':
  Model = resnet18(pretrained=True)
  num_filters = Model.fc.in_features
  print("In Features :",num_filters)
  Model.fc = nn.Sequential(
      nn.Linear(num_filters,400),
      nn.LogSoftmax(dim=1)
  )
elif config.Model=='ResNet34':
  Model = resnet34(pretrained=True)
  num_filters = Model.fc.in_features
  print("In Features :",num_filters)
  Model.fc = nn.Sequential(
      nn.Linear(num_filters,400),
      nn.LogSoftmax(dim=1)
  )
elif config.Model=='ResNet50':
  Model = resnet50(pretrained=True)
  num_filters = Model.fc.in_features
  print("In Features :",num_filters)
  Model.fc = nn.Sequential(
      nn.Linear(num_filters,400),
      nn.LogSoftmax(dim=1)
  )

elif config.Model=='ResNet101':
  Model = resnet101(pretrained=True)
  num_filters = Model.fc.in_features
  print("In Features :",num_filters)
  Model.fc = nn.Sequential(
      nn.Linear(num_filters,400),
      nn.LogSoftmax(dim=1)
  )

elif config.Model=='ResNet152':
  Model = resnet152(pretrained=True)
  num_filters = Model.fc.in_features
  print("In Features :",num_filters)
  Model.fc = nn.Sequential(
      nn.Linear(num_filters,400),
      nn.LogSoftmax(dim=1)
  )

In [ ]:
def train_log(epoch,train_loss,train_acc,test_acc,test_loss):
    wandb.log({"epoch": epoch, "train_loss": train_loss, "train_acc":train_acc,"test_loss":test_loss,"test_acc":test_acc})

In [ ]:
Model = Model.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Model.parameters(), lr=config.LEARNING_RATE)

In [ ]:
def Save_Model(model_no,model):
    model_script = torch.jit.script(Model)
    saved_path = config.saved_path + '/' + config.Model + '_' + str(model_no) +'.pt'
    model_script.save('ResNet18.pt')
    shutil.move("ResNet18.pt", saved_path)

In [ ]:
History = []
for epoch in range(config.EPOCHS):
    epoch_loss = 0
    epoch_accuracy = 0
    for data, label in train_loader:
        data = data.cuda()
        label = label.cuda()
        output = Model(data)
        #print(label.size(),output.size())
        #print(output)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in val_loader:
            data = data.cuda()
            label = label.cuda()
            val_output = Model(data)
            val_loss = criterion(val_output, label)
            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(val_loader)
            epoch_val_loss += val_loss / len(val_loader)
    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )
    train_log(epoch,epoch_loss,epoch_accuracy,epoch_val_accuracy,epoch_val_loss)
    Save_Model(epoch,Model)
    history = {
        'train_loss':epoch_loss,
        'train_acc':epoch_accuracy,
        'val_loss':epoch_val_loss,
        'val_acc':epoch_val_accuracy
    }
    History.append(history)


In [ ]:
Save_Model('final',Model)

In [ ]:
train_acc = []
train_loss = []
val_acc = []
val_loss = []
for i in range(config.EPOCHS):
  temp1 = History[i]['train_acc'].to('cpu').detach().numpy()
  temp2 = History[i]['train_loss'].to('cpu').detach().numpy()
  temp3 = History[i]['val_acc'].to('cpu').detach().numpy()
  temp4 = History[i]['val_loss'].to('cpu').detach().numpy()

  train_acc.append(temp1)
  train_loss.append(temp2)
  val_acc.append(temp3)
  val_loss.append(temp4)


plt.plot(range(config.EPOCHS),train_acc,label='Train Acc')
plt.plot(range(config.EPOCHS),val_acc,label='Val Acc')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Training and Validation Accuracy")
plt.savefig("Training.png")
plt.show()


plt.plot(range(config.EPOCHS),train_loss,label='Train Loss')
plt.plot(range(config.EPOCHS),val_loss,label='Val Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title("Training and Validation Loss")
plt.savefig("Loss.png")
plt.show()